In [ ]:
import cv2 as cv
import mediapipe as mp
import time
import numpy as np
import screen_brightness_control as sbc

In [ ]:
class BrightnessController:
    
    def __init__(self, display):
        self.display = display

    def get_brightness(self):
        # get the brightness
        return sbc.get_brightness(display=self.display)[0]
    
    def set_brightness(self, new_brightness):
        b = max(new_brightness, -10)
        b = min(b, 100)
        sbc.set_brightness(b)
        
    def modify_brightness(self, amount):
        if str(amount)[0] == '-':
            sbc.set_brightness('{}'.format(amount), display = self.display)
        else:
            sbc.set_brightness('+{}'.format(amount), display = self.display)
    
class HandPoseDetector:
    
    def __init__(self, skip_frames = 1, show_stats = False):
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands()
        self.mpDraw = mp.solutions.drawing_utils
        self.img = None
        self.resize_frac = 1
        self.hand_found = None
        self.landmarks = None
        self.skip_frames = skip_frames
        self.start_time = time.time()
        self.show_stats = show_stats
        self.tips_and_knuckles = {'index': [8,5], 'middle': [12,9], 'ring': [16,13], 'pinky': [20,17], 'thumb': [4,2]}
        self.straight_fingers = {'index': (0,False), 'middle': (0,False), 'ring': (0,False), 'pinky': (0,False), 'thumb': (0,False)}
        self.close_fingers = None
        self.size = (None, None)
        
    def _resize(self, img, interpolation = None):
        if self.resize_frac != 1:
            new_h, new_w = int(img.shape[0]*self.resize_frac), int(img.shape[1]*self.resize_frac)
            self.size = (new_h, new_w)
            if interpolation == None:
                interpolation = cv.INTER_LINEAR 
        return cv.resize(img, (new_w, new_h), interpolation= interpolation)
        
    def load_img(self, img, frac = 1):
        self.resize_frac = frac
        self.img = self._resize(img)
        
    def find_hands(self, draw = False):
        frameRGB = cv.cvtColor(self.img, cv.COLOR_BGR2RGB)
        self.hand_found = self.hands.process(frameRGB)
        if draw:
            if self.hand_found.multi_hand_landmarks:
                for handLms in self.hand_found.multi_hand_landmarks:
                    self.mpDraw.draw_landmarks(self.img, handLms, self.mpHands.HAND_CONNECTIONS)
        return self.img
    
    def load_landmarks(self, hand_number = 0, relative_coordinates = True):
        landmarks = []
        if self.hand_found.multi_hand_landmarks:
            hand = self.hand_found.multi_hand_landmarks[hand_number]
            #img_w, img_h = img.shape[0], img.shape[1]
            for id, lm in enumerate(hand.landmark):
                landmarks.append([id,lm.x, lm.y])
        self.landmarks = np.array(landmarks)
        
    def get_landmarks(self):
        return self.landmarks
    
    def get_landmark(self, number):
        return self.landmarks[self.landmarks[:,0]==number].squeeze()
    
    def draw_landmarks(self, img, all_landmarks = False, landmark_ids = None, s = 1, t = None, c = (255,0,0)):
        if self.show_stats:
            if (self.landmarks.shape[0] > 0):
                img_x, img_y = img.shape[0], img.shape[1]
                if t == None:
                    t = s*2
                if all_landmarks:
                     landmarks = self.landmarks
                else:
                    mask = np.zeros((self.landmarks.shape[0]), dtype = bool)
                    for lm in landmark_ids:
                        mask = mask + (self.landmarks[:,0] == lm)
                    landmarks = self.landmarks[mask]
                for lm in landmarks:
                        img = cv.circle(img, (int(img_y*lm[1]), int(img_x*lm[2])), s, c, t, cv.FILLED)    
        return img
    
    def _find_angle(self, p1,p2,c = np.array([0,0])):
        p1 = (p1-c)[1:]
        p2 = (p2-c)[1:]
        n1 = np.linalg.norm(p1)
        n2 = np.linalg.norm(p2)
        if n1*n2 == 0:
            return 0
        return (p1@p2)/(np.linalg.norm(p1)*np.linalg.norm(p2))

    def straight_finger(self, finger, thresh = .5):
        
        assert(finger in ['index', 'middle', 'ring', 'pinky', 'thumb'])
        tip = self.tips_and_knuckles[finger][0]
        knuckle = self.tips_and_knuckles[finger][1]
        
        straight_confidence = 0
        straight = False
        if finger == 'thumb':
            thresh = thresh *.8 # Lower threshold if the finger is thumb. 
        if (self.landmarks.shape[0] > 0):
            tip = self.get_landmark(tip)
            knuckle = self.get_landmark(knuckle)
            wrist = self.get_landmark(0)
            # Check angle with wrist
            a1 = self._find_angle(
                p1 = tip,
                p2 = knuckle,
                c = wrist)
            a1 = np.abs(a1)
            # Check distance with knucle and wrist ratio
            d1 = np.linalg.norm((tip-wrist)[1:])
            d2 = np.linalg.norm((knuckle-wrist)[1:])
            ratio = (d1/d2)
            ratio_divergence = np.abs((d1/d2)-2)
            ratio_conf = 1-ratio_divergence
            straight = (a1*ratio_conf) > thresh
            straight_confidence = np.abs(a1)*ratio_conf

        return straight_confidence, straight
    
    def close_fingertips(self, finger1, finger2, thresh = .25):
        
        assert(finger1 in ['index', 'middle', 'ring', 'pinky', 'thumb'])
        assert(finger2 in ['index', 'middle', 'ring', 'pinky', 'thumb'])
        
        close_confidence = 0 
        close = False
        
        if (self.landmarks.shape[0] > 0):
            
            # 3 point reference: Takes 3 hand landmarks as reference and calculates de max distance
            # between the points. Uses that max distance as reference to compare the distance between
            # fingertips.
            
            references = [0, 2, 9] 
            second_reference = references.copy()
            distances = []

            for r1 in references:
                r1_lm = self.get_landmark(r1)
                second_reference.remove(r1)
                for r2 in second_reference:
                    r2_lm = self.get_landmark(r2)
                    distances.append(np.linalg.norm((r1_lm-r2_lm)[1:]))
            max_dist = np.array(distances).max()

            ##

            close = False
            close_confidence = 0

            if (self.landmarks.shape[0] > 0):

                tip1_lm_number, tip2_lm_number = self.tips_and_knuckles[finger1][0], self.tips_and_knuckles[finger2][0]

                tip1 = self.get_landmark(tip1_lm_number)
                tip2 = self.get_landmark(tip2_lm_number)

                tips_distance = np.linalg.norm((tip1-tip2)[1:])

                close_confidence = 1 - tips_distance/max_dist
                close_confidence = max(close_confidence, 0)
                close_confidence = min(close_confidence, 1)
                close = (close_confidence > thresh)
            
        return close_confidence, close
    
    def find_close_fingers(self, thresh = .5):
        fingers = ['index', 'middle', 'ring', 'pinky', 'thumb']
        second_fingers = fingers.copy()
        self.close_fingers = {}
        
        for finger1 in fingers:
            second_fingers.remove(finger1)
            for finger2 in second_fingers:
                self.close_fingers[finger1+'_'+finger2] = self.close_fingertips(finger1, finger2, thresh)
        return self.close_fingers
    
    def find_straight_fingers(self, thresh = .6):
        fingers = ['index', 'middle', 'ring', 'pinky', 'thumb']
        
        for finger in fingers:
            self.straight_fingers[finger] = self.straight_finger(finger, thresh)
        return self.straight_fingers
    
    def _draw_straight_fingers(self, frame, pos = (10, 70), y_step = 30, scale = .5):
        for finger in self.straight_fingers.keys():
            extended = self.straight_fingers[finger][1]
            confidence = self.straight_fingers[finger][0]
            if extended:
                cv.putText(img = frame,
                    text = '{} ({:.2f})'.format(finger, confidence),
                    org = pos,
                    fontFace = cv.FONT_HERSHEY_COMPLEX,
                    fontScale = scale,
                    color = (0,255,0),
                    thickness = 2)
                # Update position
                pos = list(pos)
                pos[1] = pos[1] + y_step
                pos = tuple(pos)
        return frame
    
    def _draw_fps(self, frame, fontScale = 1, color = (255,0,0), thickness = 2, pos = (10, 40)):
        if self.show_stats:
            toc = time.time()
            fps = str(int(1/(toc-self.start_time)))
            self.start_time = toc
            cv.putText(img = frame,
                       text = 'FPS: {}'.format(fps),
                       org = pos,
                       fontFace = cv.FONT_HERSHEY_COMPLEX,
                       fontScale = fontScale,
                       color = color,
                       thickness = thickness)
        return frame
        
    def pose_index_middle_sticking(self):
        
        veredict = False
        
        if (self.straight_fingers['index'][1] and 
            self.straight_fingers['middle'][1] and 
            not self.straight_fingers['ring'][1] and
            not self.straight_fingers['pinky'][1] and
            not self.straight_fingers['thumb'][1]):
            
            if self.close_fingers['index_middle'][1]:
                veredict = True
        return veredict

    def draw_hand_only(self, zoom = 1, color = (0,0,255), marker_size = 2, marker_thickness = 4):
        white_canva = np.zeros((self.size[0] * zoom, self.size[1] * zoom, 3), np.uint8)
        anchor = np.array((self.size[1] * zoom , self.size[0] * zoom * 4 // 3)) // 2
        if (self.landmarks.shape[0] > 0):
            lms = self.get_landmarks()[:,1:]
            wrist = np.array([
                int(hand_controller.hand_detector.get_landmark(0)[1]*self.size[0]*zoom), 
                int(hand_controller.hand_detector.get_landmark(0)[2]*self.size[1]*zoom)])

            for lm in lms:
                pos = np.array([
                int(lm[0]*self.size[0]*zoom),
                int(lm[1]*self.size[1]*zoom)])
                
                pos = pos - wrist + anchor

                white_canva = cv.circle(white_canva, tuple(pos), marker_size, color, marker_thickness, cv.FILLED)
        
        white_canva = cv.circle(white_canva, tuple(anchor), 2, (255,255,255), 4, cv.FILLED)
        
        return white_canva
    
    def draw_hand_only_mp(self, draw = True, zoom = 1, color = (0,0,255), marker_size = 3, marker_thickness = 6):
        white_canva = np.zeros((self.size[0] * zoom, self.size[1] * zoom, 3), np.uint8)

        if draw:
            if self.hand_found.multi_hand_landmarks:
                for handLms in self.hand_found.multi_hand_landmarks:
                    self.mpDraw.draw_landmarks(white_canva, handLms, self.mpHands.HAND_CONNECTIONS)
        return white_canva
        
    
class HandBrightnessController:
    
    def __init__(self, display = 0, skip_frames = 1, show_stats = False, sensibility = 1):
        self.hand_detector = HandPoseDetector(skip_frames = skip_frames, show_stats = show_stats)
        self.brightness_controller  = BrightnessController(display = display)
        self.brightness = self.brightness_controller.get_brightness()
        self.sensibility = sensibility
        
    def test(self, webcam = 0,video_file = None, max_frames = None, all_landmarks = True, landmark_ids = []):
        if video_file:
            capture = cv.VideoCapture(video_file)
        else:
            capture = cv.VideoCapture(0)
        i = 0
        tic = 0
        isTrue, frame = capture.read()
        img_x, img_y = frame.shape[0], frame.shape[1]
        while isTrue:
            try:
                if (i % self.hand_detector.skip_frames) == 0:
                    self.hand_detector.load_img(frame, frac = .5)
                    frame_small = self.hand_detector.find_hands(draw = False)
                    self.hand_detector.load_landmarks()
                frame = self.hand_detector.draw_landmarks(frame, s = 1, all_landmarks = all_landmarks, landmark_ids = landmark_ids)
                cv.imshow('Video', frame)
                cv.waitKey(1)
                isTrue, frame = capture.read()
                i +=1
                frame = self.hand_detector._draw_fps(frame)
                self.hand_detector.find_straight_fingers(thresh = .6)
                frame = self.hand_detector._draw_straight_fingers(frame)
                _ = self.hand_detector.find_close_fingers()
                if self.hand_detector.pose_index_middle_sticking():
                    if (.4-self.hand_detector.get_landmark(8)[2] > 0):
                        modifier = 10*self.sensibility
                    else:
                        modifier = -10*self.sensibility

                    self.brightness_controller.set_brightness(modifier + self.brightness)
                    self.brightness = self.brightness_controller.get_brightness()
                    cv.putText(img = frame,
                        text = "Brightness {}".format(self.brightness),
                        org = (10, 400),
                        fontFace = cv.FONT_HERSHEY_COMPLEX,
                        fontScale = 1,
                        color = (0,0,255),
                        thickness = 2)

                if (max_frames != None):
                    if (i > max_frames):
                        print('Early stopping at {} frames.'.format(max_frames))
                        break
                cv.imshow('Canva 1', self.hand_detector.draw_hand_only_mp())
                cv.imshow('Canva', self.hand_detector.draw_hand_only())
                
            except Exception as e:
                print('Error! {}'.format(e))
                break
        capture.release()
        cv.destroyAllWindows()
    
    

In [ ]:
hand_controller = HandBrightnessController(skip_frames = 2, show_stats = True, sensibility = 0.4)

In [ ]:
hand_controller.test()